In [1]:
import os
import lean_dojo
from lean_dojo import *
from random import randint
import random
import time
import pickle
import ray
import json
import faiss
import subprocess

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-04 10:00:58,548	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
fin = open('/home/mcwave/code/automath/atp/datasets/tac_templates_in_files/train_tac_templates.json', 'r')
train_tac_templates = json.load(fin)
fin.close()
all_file_paths = list(set([x[-1] for x in train_tac_templates]))

In [3]:
# repo = LeanGitRepo(
#     "https://github.com/xiaoxin-yin/math-in-lean",
#     "20077bcd4392317ddb9605404fda3a85e40e8956"
# )

# fin = open('/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl', 'rb')
# traced_repo = pickle.load(fin)
# fin.close()

In [4]:
from utils.lean_rag_utils import *

import math
from lean_dojo import *

# Set the environment variable to disable log deduplication
os.environ['RAY_record_all_task_output'] = '1'
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Ensure Ray is not already initialized
if ray.is_initialized():
    ray.shutdown()

# Start Ray with custom configuration
ray.init(
    num_gpus=1,
    num_cpus=6,
    _memory=(192 * 1024 * 1024 * 1024),  # For example, limit Ray to 4 GB of RAM
    object_store_memory=(32 * 1024 * 1024 * 1024)  # Set object store memory to 2 GB
)


@ray.remote(num_cpus=1,num_gpus=0.16)
def compute_provability_training_data_remote(file_path, output_path):
    worker_id = random.randint(0, 100)
    cuda_version = subprocess.run(['nvcc', '--version'], capture_output=True, text=True).stdout
    print(cuda_version)
    print("CUDA:", torch.cuda.is_available())
    print("os.environ['LD_LIBRARY_PATH']", os.environ['LD_LIBRARY_PATH'])
    print("Worker", worker_id, "Loading repo...")
    repo = LeanGitRepo(
        "https://github.com/xiaoxin-yin/math-in-lean",
        "20077bcd4392317ddb9605404fda3a85e40e8956"
    )
    import pickle
    fin = open('/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl', 'rb')
    traced_repo = pickle.load(fin)
    fin.close()
    print("Worker", worker_id, "Done.")
#     theorem = Theorem(repo, "MIL/C02_Basics/solutions/Solutions_S01_Calculating.lean",
#                       "Solutions_S01_Calculating_ex2")
#     # For some theorems, it might take a few minutes.
#     dojo, state_0 = Dojo(theorem).__enter__()
#     for i in range(100000000):
#         state_1 = dojo.run_tac(state_0, "rw [← mul_assoc]")
#         state_2 = dojo.run_tac(state_1, "rw [mul_comm a]")
#         if i % 1000 == 0:
#             print("Worker", worker_id, "finishes", i)
    #
    print("Worker", worker_id, "Loading models...")
    # Load pre-trained BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model_state = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_state-batch64-60k-loss0047.model')
    #model_tac   = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_tac-batch64-60k-loss0047.model')
    device = model_state.device
    print(device)
    #
    print("Worker", worker_id, "Loading faiss index ...")
    index = faiss.read_index('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/faiss_index_bert_embeds-batch64-60k-loss0047.idx')
    fin = open('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/tac_template_freq.json', 'r')
    tac_template_freq = json.load(fin)
    fin.close()
    tacs = list(tac_template_freq.keys())
    #
    print("Worker", worker_id, "Computing provability ...")
    compute_provability_training_data(file_path,
                                      output_path,
                                      traced_repo,
                                      model_state,
                                      tokenizer,
                                      index,
                                      tacs)

all_file_paths = [
    "MIL/C02_Basics/solutions/Solutions_S01_Calculating.lean",
    "MIL/C02_Basics/solutions/Solutions_S02_Proving_Identities_in_Algebraic_Structures.lean",
    "MIL/C02_Basics/solutions/Solutions_S03_Using_Theorems_and_Lemmas.lean",
    "MIL/C02_Basics/solutions/Solutions_S04_More_on_Order_and_Divisibility.lean",
    "MIL/C02_Basics/solutions/Solutions_S05_Proving_Facts_about_Algebraic_Structures.lean"
]    
output_folder = '/home/mcwave/code/automath/atp/datasets/provability/rag/'

# Submit tasks
result_ids = [compute_provability_training_data_remote.remote(file_path, output_folder) for file_path in all_file_paths]

# Fetch results
results = ray.get(result_ids)

# Optionally, shut down Ray if you're done with all computations
ray.shutdown()

# Display results
print(results)

2024-06-04 10:01:02,591	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(compute_provability_training_data_remote pid=394980) nvcc: NVIDIA (R) Cuda compiler driver
(compute_provability_training_data_remote pid=394980) Copyright (c) 2005-2023 NVIDIA Corporation
(compute_provability_training_data_remote pid=394980) Built on Tue_Jun_13_19:16:58_PDT_2023
(compute_provability_training_data_remote pid=394980) Cuda compilation tools, release 12.2, V12.2.91
(compute_provability_training_data_remote pid=394980) Build cuda_12.2.r12.2/compiler.32965470_0
(compute_provability_training_data_remote pid=394980) 
(compute_provability_training_data_remote pid=394980) CUDA: True
(compute_provability_training_data_remote pid=394980) os.environ['LD_LIBRARY_PATH'] /usr/local/cuda-12.2/lib64:
(compute_provability_training_data_remote pid=394980) Worker 92 Loading repo...
(compute_provability_training_data_remote pid=394978) Worker 54 Done.
(compute_provability_training_data_remote pid=394978) Worker 54 Loading models...
(compute_provability_training_data_remote pid=394978) nvcc

(compute_provability_training_data_remote pid=394978) 2024-06-04 10:03:11.371 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=394978) Worker 54 Computing provability ...
(compute_provability_training_data_remote pid=394978) THEOREM: Solutions_S05_Proving_Facts_about_Algebraic_Structures_ex1
(compute_provability_training_data_remote pid=394978) theorem Solutions_S05_Proving_Facts_about_Algebraic_Structures_ex1 : x ⊓ y = y ⊓ x
(compute_provability_training_data_remote pid=394978) theorem loaded
(compute_provability_training_data_remote pid=394980) theorem Solutions_S01_Calculating_ex1 (a b c : ℝ) : c * b * a = b * (a * c)
(compute_provability_training_data_remote pid=394979) theorem Solutions_S03_Using_Theorems_and_Lemmas_ex1 (h₀ : a ≤ b) (h₁ : b < c) (h₂ : c ≤ d) (h₃ : d < e) : a < e
(compute_provability_training_data_remote pid=394982) theorem Solutions_S04_More_on_Order_and_Divisibility_ex1 : max a b = max b a
(compute_provability_training_data_remote pid=394981) THEOREM: MyRing.add_neg_cancel_right
(compute_provability_training_data_remote pid=394981) theorem ad

(compute_provability_training_data_remote pid=394978) 2024-06-04 10:05:22.805 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely. [repeated 5x across cluster]


...

(compute_provability_training_data_remote pid=394979) 2024-06-04 10:11:35.667 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=394981) 78 ancestors found [repeated 162x across cluster]
(compute_provability_training_data_remote pid=394981) ProofFinished(tactic_state_id=5251, message='') [repeated 162x across cluster]


(compute_provability_training_data_remote pid=394978) 2024-06-04 10:11:50.562 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=394978) 1984 states in total
(compute_provability_training_data_remote pid=394978) 8 proven states
(compute_provability_training_data_remote pid=394978) 94 state pairs found
(compute_provability_training_data_remote pid=394978) THEOREM: Solutions_S05_Proving_Facts_about_Algebraic_Structures_ex4
(compute_provability_training_data_remote pid=394978) theorem Solutions_S05_Proving_Facts_about_Algebraic_Structures_ex4 : x ⊔ y ⊔ z = x ⊔ (y ⊔ z)
(compute_provability_training_data_remote pid=394978) theorem loaded
(compute_provability_training_data_remote pid=394978) 6 ancestors found [repeated 87x across cluster]
(compute_provability_training_data_remote pid=394978) ProofFinished(tactic_state_id=2709, message='') [repeated 87x across cluster]



KeyboardInterrupt



In [5]:
import ray

# Ensure Ray is not already initialized
if ray.is_initialized():
    ray.shutdown()

In [ ]:
import subprocess

print(torch.cuda.is_available())
print(os.environ['LD_LIBRARY_PATH'])
cuda_version = subprocess.run(['nvcc', '--version'], capture_output=True, text=True).stdout
print(cuda_version)